# Create TensorFlow Session

In [1]:
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


# Define Model

In [2]:
from keras.models import Model
from keras.layers import Input, TimeDistributed, Flatten, RepeatVector, SimpleRNN, Reshape, UpSampling2D
from keras.layers.merge import Multiply
from keras.layers import Conv2D, Permute, MaxPooling1D, Activation
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np
import keras.backend as K
from keras.constraints import nonneg
from keras.regularizers import l1

B, T, H, W, C, D = 90, 2, 28, 28, 1, 10

img = Input(batch_shape=[B, H, W*T, C])
zero = Input(batch_shape=[B, 1])

with tf.name_scope('Repeat'):
    flat_img = Flatten()(img)
    zeros = RepeatVector(n=T)(zero)
    flat_imgs = RepeatVector(n=T)(flat_img)
    imgs = Reshape([T, H, W*T, C], name='Images')(flat_imgs)

with tf.name_scope('Mask'):
    small_flat_masks = SimpleRNN(units=T,
                                 return_sequences=True,
                                 use_bias=False,
                                 stateful=True,
                                 name='recurrent',
                                 kernel_regularizer=l1(1e-4))(zeros)
    small_flat_masks = Reshape([T, 1, T, C])(small_flat_masks)
    masks = TimeDistributed(UpSampling2D(size=[H, W]), name='Masks')(small_flat_masks)

masked_imgs = Multiply()([imgs, masks])

with tf.name_scope('Convolution'):
    matched = TimeDistributed(Conv2D(filters=D,
                                     kernel_size=[H, W],
                                     strides=[1, W],
                                     kernel_constraint=nonneg()))(masked_imgs)
    matched = Reshape([T, T, D])(matched)

with tf.name_scope('MaxPooling'):
    pooled = TimeDistributed(MaxPooling1D(pool_size=T))(matched)
    scores = Reshape([T, D])(pooled)
    
with tf.name_scope('Softmax'):
    probs = TimeDistributed(Activation('softmax'))(scores)

model = Model(inputs=[img, zero], outputs=probs)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (90, 1)               0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (90, 28, 56, 1)       0                                            
____________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)   (90, 2, 1)            0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (90, 1568)            0                                            
___________________________________________________________________________________________

# Visualize Model

1. TensorBoard
2. Keras Layers

In [3]:
from support import show_graph

show_graph(sess.graph)
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

# Load MNIST Digits

In [4]:
from IPython.display import display
from keras.preprocessing.image import array_to_img
import numpy as np
from keras.datasets import mnist

[X_mnist, y_mnist], _ = mnist.load_data()
X_mnist = X_mnist/255.

digit_idxs = [np.argwhere(y_mnist == d).flatten()[0] for d in range(10)]
X_mnist = X_mnist[digit_idxs]
for x in X_mnist:
    img_ = array_to_img(np.expand_dims(x, axis=0), data_format='channels_first')
    display(img_)

X_mnist.shape

(10, 28, 28)

# Create Images and Labels

In [5]:
import itertools
from keras.preprocessing.image import array_to_img

nb_train = 90
X, y = np.zeros([nb_train, H, W*T]), [None]*nb_train
i = 0
D = set()
while i < nb_train:
    digits = np.random.choice(10, size=T, replace=False)
    if tuple(digits) in D:
        continue
        
    D.add(tuple(digits))
    X[i] = np.hstack([x for x in X_mnist[digits]])
    y[i] = digits
    i += 1

for idx in range(5):
    x = X[idx]
    img_ = array_to_img(np.expand_dims(X[idx], axis=-1))
    print(y[idx])
    display(img_)

[6 8]


[3 5]


[2 5]


[4 8]


[3 1]


# Make Labels Categorical

In [6]:
import numpy as np

Y = np.zeros([nb_train, T, 10])
for i, ys in enumerate(y):
    for j, y_ in enumerate(ys):
        Y[i, j, y_] = 1
    
Y.shape

(90, 2, 10)

# Fit Model

In [7]:
import keras

class ResetStates(keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        h0 = np.zeros([1, T])
        h0[:, -1] = 1
        h0 = h0.flatten()
        self.h0 = np.repeat(np.expand_dims(h0, axis=0), repeats=B, axis=0)
        
    def on_batch_begin(self, batch, logs={}):
        self.model.get_layer('recurrent').reset_states(states_value=self.h0)
    
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_ = np.expand_dims(X, axis=-1) # add color channel
model.fit([X_, np.zeros([nb_train, 1])], Y,
          epochs=100,
          callbacks=[ResetStates()],
          batch_size=B)

Epoch 1/100


InternalError: Blas SGEMM launch failed : a.shape=(180, 1), b.shape=(1, 2), m=180, n=2, k=1
	 [[Node: Mask/recurrent/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](Mask/recurrent/Reshape, Mask/recurrent/kernel/read)]]
	 [[Node: add/_39 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_451_add", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Mask/recurrent/MatMul', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/applic

# Visualize Convolutional Filters

In [43]:
W, b = model.layers[10].get_weights()

for i in range(10):
    w = W[..., i]
    img_ = array_to_img(w)
    display(img_)

# Get Masks and Masked Images

In [44]:
model_ = Model(inputs=[img, zero], outputs=[masks, masked_imgs])

h0 = np.zeros([1, T])
h0[:, -1] = 1
h0 = h0.flatten()
h0 = np.repeat(np.expand_dims(h0, axis=0), repeats=B, axis=0)
        
model_.get_layer('recurrent').reset_states(states_value=h0)
H, I = model_.predict([X_, np.zeros([nb_train, 1])], batch_size=B)

# Visualize Masks and Masked Images

In [45]:
idx = np.random.choice(nb_train)

for h, i in zip(H[idx], I[idx]):
    mask, masked_img = array_to_img(h), array_to_img(i)
    display(mask)
    display(masked_img)

In [46]:
import seaborn as sns
import pandas as pd

W, b = model.get_layer('recurrent').get_weights()

pd.DataFrame(b).style\
.background_gradient(cmap=sns.diverging_palette(250, 5, as_cmap=True))